# Programming task

**The task should be completed with the current values of the hyperparameters. For verification, below you will see the answers that should be obtained. After all the answers match, you can use the resulting notebook to complete your individual task.**

Set model hyperparameters

In [3]:
epsilon = 0.1 # Epsilon parameter which is used in epsilon-greedy strategy
gamma = 0.8 # Discount coefficient gamma
random_seed = 100 #Random seed
time_delay = 1 # Time delay when rendering the game process after training (seconds)
lr_rate = 0.9 #Learning rate alpha

We import the libraries, create our own 6x6 environment. S denotes the starting point. F - ice is safe (frozen), H - hole, G - goal. The parameter `is_slippery = False` is responsible for slipping. That is, if the agent chose the action to go right, then it will move to the corresponding state. In the general case, due to the “slipping”, one may find itself in a different state. We also copied from the GYM library and slightly modified the function `` generate_random_map ``, in order to generate arbitrary maps based on `` random_seed ``.

In [1]:
!git clone https://github.com/dvolchek/gym_0_18_0.git -q
%cd /content/gym_0_18_0
!pip install -e. -q


import gym
import numpy as np
import time
from IPython.display import clear_output


def generate_random_map(size, p, sd):
    """Generates a random valid map (one that has a path from start to goal)
    :param size: size of each side of the grid
    :param p: probability that a tile is frozen
    """
    valid = False
    np.random.seed(sd)

    # DFS to check that it's a valid path.
    def is_valid(res):
        frontier, discovered = [], set()
        frontier.append((0,0))
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == 'G':
                        return True
                    if (res[r_new][c_new] not in '#H'):
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(['F', 'H'], (size, size), p=[p, 1-p])
        res[0][0] = 'S'
        res[-1][-1] = 'G'
        valid = is_valid(res)
    return ["".join(x) for x in res]

# Map generation
random_map = generate_random_map(size=6, p=0.8, sd = random_seed) #Create our map
env = gym.make("FrozenLake-v0", desc=random_map, is_slippery=False) #Initialize environment
print("Your map")
env.render() #Render the map

[Errno 2] No such file or directory: '/content/gym_0_18_0'
/Users/roman14/Desktop/Pycharm/ITMO_labs/BigData_ML_online
ERROR: file:///Users/roman14/Desktop/Pycharm/ITMO_labs/BigData_ML_online does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


NameError: name 'random_seed' is not defined

Functions for selecting an action and updating an action value table. The line *** is used to check responses in openedx. Outside of the academic problem, it is better to use the original method of the `environment` class, that is:

`action = env.action_space.sample ()`

# Task 1
Complete the function `` learn () ``, so that as a result of its call, the value of the current action is updated according to the Q-learning algorithm

In [ ]:
def choose_action(state):
    action=0
    if np.random.uniform(0, 1) < epsilon:
        action = np.random.randint(0,env.action_space.n) #***
    else:
        action = np.random.choice(np.array(np.argwhere(Q[state, :] == np.amax(Q[state, :])).flatten().tolist()))
    return action

def learn(state, state2, reward, action, done):
    #Q[state, action] = #Your code here
    # Оценка ценности текущего состояния и действия
    q_value = Q[state, action]

    if not done:
        # Оценка ценности следующего состояния
        next_max = np.max(Q[state2])
        # Обновление ценности текущего действия
        q_target = reward + gamma * next_max
    else:
        # Если достигнуто конечное состояние, обновление ценности равно только полученному вознаграждению
        q_target = reward

    # Обновление ценности текущего действия
    Q[state, action] += lr_rate * (q_target - q_value)

# Task 2
Complete the following code so that as a result of training the model you could find out the number of wins and the number of the game (`game`), on which the agent won the fifth victory in a row for the first time.

Let's explain what the function ```env.step(action)``` returns

```state2``` --  next state

```reward``` -- reward

```done``` -- inidcator of the end of the game. True in case of victory or fall into the hole. False in other cases.




In [ ]:
from tqdm import tqdm
# Inititalization
np.random.seed(random_seed)
total_games = 10000
max_steps = 100
Q = np.zeros((env.observation_space.n, env.action_space.n))
#Main cycle
for game in tqdm(range(total_games)):
    state = env.reset()
    t = 0
    while t < max_steps:

        t += 1

        action = choose_action(state)

        state2, reward, done, info = env.step(action)

        if t == max_steps:
          done = True

        learn(state, state2, reward, action, done)

        state = state2

        if done:
          break


Output answers with the given parameters

In [ ]:
print("The number of victories in a series of 10,000 games: ", #Your code here)
print("Five wins in a row were first won in the game ", #Your code here)

In [2]:
from tqdm import tqdm
import numpy as np

# Inititalization
np.random.seed(random_seed)
total_games = 10000
max_steps = 100
Q = np.zeros((env.observation_space.n, env.action_space.n))
wins_in_series = 0
first_five_wins_game = None
total_wins = 0

#Main cycle
for game in tqdm(range(total_games)):
    state = env.reset()
    t = 0
    while t < max_steps:
        t += 1
        action = choose_action(state)
        state2, reward, done, info = env.step(action)
        if t == max_steps:
            done = True
        learn(state, state2, reward, action, done)
        state = state2
        if done:
            if reward == 1:  # Победа
                total_wins += 1
                wins_in_series += 1
                if wins_in_series == 5 and first_five_wins_game is None:
                    first_five_wins_game = game
            else:
                wins_in_series = 0  # Сбрасываем счетчик серии побед
            break

print("Количество побед в серии из 10 000 игр:", total_wins)
print("Пять побед подряд впервые было одержано в игре", first_five_wins_game+1)

NameError: name 'random_seed' is not defined

The following results should be obtained.


*  The number of victories in a series of 10,000 games:  7914
*  Five wins in a row were first won in the game  885




Let's perform one game to track the actions of the agent. At the same time, we will consider the model fully trained, that is, actions are selected according to the greedy strategy, the values of the actions in the table are not updated.

In [ ]:
import time
#Greedy action selection
def choose_action_one_game(state):
    action = np.random.choice(np.array(np.argwhere(Q[state, :] == np.amax(Q[state, :])).flatten().tolist()))
    return action

states=[]#Array to save agent states during the game
t = 0
state = env.reset()
wn = 0
while(t<100):
  env.render()
  time.sleep(time_delay)
  clear_output(wait=True)
  action = choose_action_one_game(state)
  state2, reward, done, info = env.step(action)
  states.append(state)
  state = state2
  t += 1
  if done and reward == 1:
    wn=1
  if done:
    break
if wn == 1:
  print("!!!WIN!!!")

Route map

In [ ]:
import matplotlib.pyplot as plt

def make_maze_pic(maze):
  maze_pic=[]
  for i in range(len(maze)):
    row = []
    for j in range(len(maze[i])):
      if maze[i][j] == 'S':
        row.append(0)
      if maze[i][j] == 'F':
        row.append(0)
      if maze[i][j] == 'H':
        row.append(1)
      if maze[i][j] == 'G':
        row.append(0)
    maze_pic.append(row)
  maze_pic = np.array(maze_pic)
  return maze_pic


#Make maze fit to plot
maze_pic = make_maze_pic(random_map)
nrows, ncols = maze_pic.shape

#Arrays of picture elements
rw = np.remainder(states,nrows)
cl = np.floor_divide(states,nrows)
if wn == 1:
  rw = np.append(rw, [nrows-1])
  cl = np.append(cl,[ncols-1])

#Picture plotting
fig, ax1 = plt.subplots(1, 1, tight_layout=True)
ax1.clear()
ax1.set_xticks(np.arange(0.5, nrows, step=1))
ax1.set_xticklabels([])
ax1.set_yticks(np.arange(0.5, ncols, step=1))
ax1.set_yticklabels([])
ax1.grid(True)
ax1.plot([0],[0], "gs", markersize=40)  # start is a big green square
ax1.text(0, 0.2,"Start", ha="center", va="center", color="white", fontsize=12) #Start text
ax1.plot([nrows-1],[ncols-1], "rs", markersize=40)  # exit is a big red square
ax1.text(nrows-1, ncols-1+0.2,"Finish", ha="center", va="center", color="white", fontsize=12) #Exit text
ax1.plot(rw,cl, ls = '-', color = 'blue') #Blue lines path
ax1.plot(rw,cl, "bo")  # Blue dots visited cells
ax1.imshow(maze_pic, cmap="binary")